# Práctica 1: Buscador equivalencias fonéticas en un corpus

In [1]:
import requests as r
import difflib

In [2]:
lang_codes = {
  "ar": "Arabic (Modern Standard)",
  "de": "German",
  "en_UK": "English (Received Pronunciation)",
  "en_US": "English (General American)",
  "eo": "Esperanto",
  "es_ES": "Spanish (Spain)",
  "es_MX": "Spanish (Mexico)",
  "fa": "Persian",
  "fi": "Finnish",
  "fr_FR": "French (France)",
  "fr_QC": "French (Québec)",
  "is": "Icelandic",
  "ja": "Japanese",
  "jam": "Jamaican Creole",
  "km": "Khmer",
  "ko": "Korean",
  "ma": "Malay (Malaysian and Indonesian)",
  "nb": "Norwegian Bokmål",
  "nl": "Dutch",
  "or": "Odia",
  "ro": "Romanian",
  "sv": "Swedish",
  "sw": "Swahili",
  "tts": "Isan",
  "vi_C": "Vietnamese (Central)",
  "vi_N": "Vietnamese (Northern)",
  "vi_S": "Vietnamese (Southern)",
  "yue": "Cantonese",
  "zh": "Mandarin"
}
iso_lang_codes = list(lang_codes.keys())

Algunas funciones auxiliares

In [3]:
def response_to_dict(ipa_list: list) -> dict:
    """Parse to dict the list of word-IPA

    Each element of text hae the format:
    [WORD][TAB][IPA]

    Parameters
    ----------
    ipa_list: list
        List with each row of ipa-dict raw dataset file

    Returns
    -------
    dict:
        A dictionary with the word as key and the phonetic
        representation as value
    """
    result = {}
    for item in ipa_list:
        item_list = item.split("\t")
        result[item_list[0]] = item_list[1]
    return result

def get_ipa_transcriptions(word: str, dataset: dict) -> list[str]:
    """Search for word in a given dataset of IPA phonetics

    Given a word this function return the IPA transcriptions

    Parameters:
    -----------
    word: str
        A word to search in the dataset
    dataset: dict
        A dataset for a given language code
    Returns
    -------
    """
    return dataset.get(word.lower(), "NOT FOUND").split(", ")

## Función para traer los diccionarios usando una "cache"

Optimizar el código para agregar los datasets en un "cache" a demanda y no descargar todo el corpus de un trancazo. Esto quiere decir que al inicio de la ejecución no habrá ningun dataset descargado. Mientras la usuaria vaya seleccionado idiomas los irá agregando a un cache local (puede ser persistente o en memoria). Ejemplo:

```
lang>> es_MX
Corpus no encontrado. Descargando ...
[es_MX]>>
...
lang>> en_US
Corpus no encontrado. Descargando ...
[en_US]>>
...
lang>> es_MX
[es_MX]>>
...
````

In [4]:
# Diccionario en memoria para almacenar los datasets descargados
cache = {}

def get_ipa_dict_cached(iso_lang: str) -> dict:
    """Obtener archivo ipa-dict desde Github o caché local.

    Parámetros:
    -----------
    iso_lang:
        Lenguaje como código iso

    Resultados:
    --------
    dict:
        Diccionario con palabras como claves y representación fonética
        como valores para un código de idioma dado.
    """
    if iso_lang in cache:
        print(f"Usando caché para {iso_lang}")
        return cache[iso_lang]
    
    print(f"Corpus para {iso_lang} no encontrado. Descargando...")
    response = r.get(f"https://raw.githubusercontent.com/open-dict-data/ipa-dict/master/data/{iso_lang}.txt")
    if response.status_code == 200:
        raw_data = response.text.split("\n")
        data_dict = response_to_dict(raw_data[:-1])
        cache[iso_lang] = data_dict
        print("... listo")
        return data_dict
    else:
        print(f"Corpus no encontrado para {iso_lang}.")
        return {}

Función para que el usuario ingrese lenguajes válidos:

In [5]:
def get_valid_language():
    print(f"Elige uno de los siguientes idiomas: {(iso_lang_codes)}")
    print("Para salir sólo presiona enter")

    lang = input("lang>> ")
    while lang:
        if lang in lang_codes:
            return lang
        else:
            lang = input("lang>> ")
    return lang

## Modo de búsqueda por frases

Agregar un nuevo modo de búsqueda donde se extienda el comportamiento básico del buscador para ahora buscar por frases. Ejemplo:

```
[es_MX]>>> Hola que hace
 /ola/ /ke/ /ase/
```

In [6]:
def phrase_to_ipa(phrase: str, dataset: dict) -> str:
    """Get the IPA representation of a phrase

    Given a string, split it in words (by whitespaces), find the IPA representation of 
    each word (only take the first representation), and return a new string replacing
    each word in te oroginal phrase with its IPA translation.

    Returns
    -------
    str
        Representation in IPA of the original phrase
    """
    words = phrase.split(' ')
    ipa_phrase = ''
    for word in words:
        ipa_phrase += get_ipa_transcriptions(word, dataset)[0]
        ipa_phrase += ' '
    return ipa_phrase.rstrip()

In [7]:
print("Representación fonética de frases")

lang = get_valid_language()
print(f"Selected language: {lang_codes[lang]}") if lang else print("Adios 👋🏼")
while lang:
    sub_dataset = get_ipa_dict_cached(lang)
    query = input(f"  [{lang}]word>> ")
    print(query, " | ", phrase_to_ipa(query, sub_dataset))
    while query:
        query = input(f"  [{lang}]word>> ")
        print(query, " | ", phrase_to_ipa(query, sub_dataset))
    lang = get_valid_language()
    print(f"Selected language: {lang_codes[lang]}") if lang else print("Adios 👋🏼")

Representación fonética de frases
Elige uno de los siguientes idiomas: ['ar', 'de', 'en_UK', 'en_US', 'eo', 'es_ES', 'es_MX', 'fa', 'fi', 'fr_FR', 'fr_QC', 'is', 'ja', 'jam', 'km', 'ko', 'ma', 'nb', 'nl', 'or', 'ro', 'sv', 'sw', 'tts', 'vi_C', 'vi_N', 'vi_S', 'yue', 'zh']
Para salir sólo presiona enter


lang>>  hola que hace
lang>>  


Adios 👋🏼


## Modo de búsqueda normal con sugerencia de palabras similares

Mejorar la solución al escenario cuando no se encuentran las palabras en el dataset mostrando palabras similares. Este modo sólo acepta palabras, no frases. Ejemplo:

```
[es_MX]>> pero
No se encontro <<pero>> en el dataset. Palabras aproximadas:
perro /pero/
perno /peɾno/
[es_MX]>>
```

In [10]:
def find_similar_words(input_word, dataset, threshold=0.6, amount=5):
    """Get list of similar words

    Return a list of the best “good enough” matches of string input_word in the given dataset.

    Returns
    -------
    list[str]
        RList of similar words
    """
    similar_words = difflib.get_close_matches(input_word, dataset.keys(), n=amount, cutoff=threshold)
    return similar_words

def print_word_transcription(input_word, dataset):
    """ Prints the transcription results for the given input_word and dataset """
    results = get_ipa_transcriptions(input_word, dataset)
    print(input_word, " | ", ", ".join(results))
    
def search_word(input_word, dataset):
    """ 
    Searches for the given input_word in the dataset, and depending on whether the word is found
    or not, prints the IPA transcriptions of the word, or of its similar words.
    """
    if input_word in dataset:
        print_word_transcription(input_word, dataset)
    else:
        print(f"No se encontró {input_word} en el dataset. Buscando palabras aproximadas...")
        similar_words = find_similar_words(input_word, dataset)
        if len(similar_words)>0:
            print("Palabras aproximadas:")
            for word in similar_words:
                print_word_transcription(word, dataset)
        else:
            print("No se encontraron palabras parecidas.")

In [11]:
print("Representación fonética de palabras")

lang = get_valid_language()
print(f"Selected language: {lang_codes[lang]}") if lang else print("Adios 👋🏼")
while lang:
    dataset = get_ipa_dict_cached(lang)
    query = input(f"  [{lang}]word>> ")
    search_word(query, dataset)
    while query:
        query = input(f"  [{lang}]word>> ")
        search_word(query, dataset)
    lang = get_valid_language()
    print(f"Selected language: {lang_codes[lang]}") if lang else print("Adios 👋🏼")

Representación fonética de palabras
Elige uno de los siguientes idiomas: ['ar', 'de', 'en_UK', 'en_US', 'eo', 'es_ES', 'es_MX', 'fa', 'fi', 'fr_FR', 'fr_QC', 'is', 'ja', 'jam', 'km', 'ko', 'ma', 'nb', 'nl', 'or', 'ro', 'sv', 'sw', 'tts', 'vi_C', 'vi_N', 'vi_S', 'yue', 'zh']
Para salir sólo presiona enter


lang>>  de


Selected language: German
Corpus para de no encontrado. Descargando...
... listo


  [de]word>>  hallo


hallo  |  /haˈloː/, /ˈhalo/


...

No se encontró halllo en el dataset. Buscando palabras aproximadas...
Palabras aproximadas:
hallo  |  /haˈloː/, /ˈhalo/
hall  |  /hal/
haltlos  |  /ˈhaltloːs/
halo-  |  /halo/
hallt  |  /halt/


  [de]word>>  


No se encontró  en el dataset. Buscando palabras aproximadas...
No se encontraron palabras parecidas.
Elige uno de los siguientes idiomas: ['ar', 'de', 'en_UK', 'en_US', 'eo', 'es_ES', 'es_MX', 'fa', 'fi', 'fr_FR', 'fr_QC', 'is', 'ja', 'jam', 'km', 'ko', 'ma', 'nb', 'nl', 'or', 'ro', 'sv', 'sw', 'tts', 'vi_C', 'vi_N', 'vi_S', 'yue', 'zh']
Para salir sólo presiona enter


lang>>  


Adios 👋🏼
